## Load Packages

In [ ]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm 

## Working of NER

In [ ]:
# nlp1 = spacy.load('en_core_web_sm')
# docx1 = nlp1(u"Who is Nishanth?")

In [ ]:
# for token in docx1.ents:
#     print(token.text,token.start_char, token.end_char,token.label_)

In [ ]:
# docx2 = nlp1(u"Who is Kamal Khumar?")

In [ ]:
# for token in docx2.ents:
#     print(token.text,token.start_char, token.end_char,token.label_)

## Train and Test Data

In [ ]:
TRAIN_DATA = [
    ('I am taking cosc 301', {
        'entities': [(12, 20, 'COR')]
    }),
    ('I am taking phil 331', {
        'entities': [(12, 20, 'COR')]
    }),
    ('I am taking data 301', {
        'entities': [(12, 20, 'COR')]
    }),
    ('I love engl 351 so much', {
        'entities': [(7, 15, "COR")]
    }),
    ('Why is span 419 so hard', {
        'entities': [(7, 15, "COR")]
    }),
    ('I just enrolled in math 101 and I am excited to start learning about calculus.', {
    'entities': [(20, 28, "COR")]
    }),
    ('Last semester, I took psyc 200 and it was one of my favorite classes.', {
        'entities': [(18, 26, "COR")]
    }),
    ('psyc 200 is one of my favorite classes.', {
        'entities': [(0, 7, "COR")]
    }),
    ('I am struggling in econ 301, but I am determined to improve my understanding of macroeconomics.', {
        'entities': [(19, 27, "COR")]
    }),
    ('I am considering taking chem 110 next semester.', {
        'entities': [(25, 33, "COR")]
    }),
    ('I have a final exam in stat 400 next week and I have been studying non-stop to prepare.', {
        'entities': [(22, 30, "COR")]
    }),
    ('I am struggling in mus 210, but I am determined to improve my musical abilities.', {
        'entities': [(19, 26, "COR")]
    }),
    
    # Computer Science
    ('I am majoring in computer science', {'entities': [(18, 34, 'MAJOR')]}),
    ('My major is computer science', {'entities': [(12, 28, 'MAJOR')]}),
#     ('I study computer science', {'entities': [(7, 23, 'MAJOR')]}),
#     ('Computer science is my field of study', {'entities': [(0, 17, 'MAJOR')]}),
    
    # Biology
    ('I am majoring in biology', {'entities': [(18, 25, 'MAJOR')]}),
    ('My major is biology', {'entities': [(12, 19, 'MAJOR')]}),
#     ('I study biology', {'entities': [(7, 14, 'MAJOR')]}),
#     ('Biology is my field of study', {'entities': [(0, 7, 'MAJOR')]}),
    
    # Data Science
#     ('I am majoring in data science', {'entities': [(18, 30, 'MAJOR')]}),
#     ('My major is data science', {'entities': [(12, 24, 'MAJOR')]}),
#     ('I study data science', {'entities': [(7, 19, 'MAJOR')]}),
#     ('Data science is my field of study', {'entities': [(0, 12, 'MAJOR')]}),
    
    # Mechanical Engineering
    ('I am majoring in mechanical engineering', {'entities': [(18, 40, 'MAJOR')]}),
    ('My major is mechanical engineering', {'entities': [(12, 34, 'MAJOR')]}),
#     ('I study mechanical engineering', {'entities': [(7, 32, 'MAJOR')]}),
#     ('Mechanical engineering is my field of study', {'entities': [(0, 23, 'MAJOR')]}),
    
    # Physics
    ('I am majoring in physics', {'entities': [(18, 25, 'MAJOR')]}),
    ('My major is physics', {'entities': [(12, 19, 'MAJOR')]}),
#     ('I study physics', {'entities': [(7, 14, 'MAJOR')]}),
#     ('Physics is my field of study', {'entities': [(0, 7, 'MAJOR')]}),
    
    # Psychology
    ('I am majoring in psychology', {'entities': [(18, 28, 'MAJOR')]}),
    ('My major is psychology', {'entities': [(12, 22, 'MAJOR')]}),
#     ('I study psychology', {'entities': [(7, 17, 'MAJOR')]}),
#     ('Psychology is my field of study', {'entities': [(0, 10, 'MAJOR')]}),
    
    # Business
#     ('I am majoring in business', {'entities': [(18, 25, 'MAJOR')]}),
#     ('My major is business', {'entities': [(12, 19, 'MAJOR')]}),
#     ('I study business', {'entities': [(7, 14, 'MAJOR')]}),
#     ('Business is my field of study', {'entities': [(0, 7, 'MAJOR')]}),
    
    # Chemistry
    ('I am majoring in chemistry', {'entities': [(18, 27, 'MAJOR')]}),
    ('My major is chemistry', {'entities': [(12, 21, 'MAJOR')]}),
    ('I study chemistry', {'entities': [(7, 16, 'MAJOR')]}),
    ('Chemistry is my field of study', {'entities': [(0, 9, 'MAJOR')]}),

    # Mathematics
    ('I am majoring in mathematics', {'entities': [(18, 29, 'MAJOR')]}),
    ('My major is mathematics', {'entities': [(12, 23, 'MAJOR')]}),
#     ('I study mathematics', {'entities': [(7, 18, 'MAJOR')]}),
#     ('Mathematics is my field of study', {'entities': [(0, 12, 'MAJOR')]}),

    # English
#     ('I am majoring in English', {'entities': [(18, 24, 'MAJOR')]}),
#     ('My major is English', {'entities': [(12, 18, 'MAJOR')]}),
#     ('I study English', {'entities': [(7, 13, 'MAJOR')]}),
#     ('English is my field of study', {'entities': [(0, 7, 'MAJOR')]}),

    # Environmental Science
#     ('I am majoring in environmental science', {'entities': [(18, 39, 'MAJOR')]}),
#     ('My major is environmental science', {'entities': [(12, 33, 'MAJOR')]}),
#     ('I study environmental science', {'entities': [(7, 28, 'MAJOR')]}),
#     ('Environmental science is my field of study', {'entities': [(0, 22, 'MAJOR')]}),

    # History
#     ('I am majoring in history', {'entities': [(18, 25, 'MAJOR')]}),
#     ('My major is history', {'entities': [(12, 19, 'MAJOR')]}),
#     ('I study history', {'entities': [(7, 14, 'MAJOR')]}),
#     ('History is my field of study', {'entities': [(0, 7, 'MAJOR')]})
]

TEST_DATA = [
     ('How is math 101?', {
        'entities': [(7, 15, 'PER')]
    }),
     ('Who is cosc 554?', {
        'entities': [(7, 19, 'PER')]
    }),
    ('I like London and Berlin.', {
        'entities': [(7, 13, 'LOC'), (18, 24, 'LOC')]
    }),
    ('what is phil 331 about', {
        'entities': [(12, 20, 'COR')]
    })
]

TEST_Sentences = ["I am taking COSC 301", "I'm taking PSYC 210 and learning about the different approaches to psychology.",    "I just enrolled in math 101 and I'm excited to start learning about calculus.",    "I'm considering taking MATH 202 next semester, but I'm not sure if I'm ready for the challenge.",    "Last semester, I took PSYC 200 and it was one of my favorite classes.",    "I'm struggling in ECON 301, but I'm determined to improve my understanding of macroeconomics.",    "This summer, I'm enrolled in SPAN 301 and looking forward to improving my language skills.",    "I have a final exam in STAT 400 next week and I have been studying non-stop to prepare.",    "I loved taking HIST 205 and learning about the Civil Rights Movement.",    "I am considering taking PHIL 331 next semester to fulfill my ethics requirement.", "I'm taking ART 101 this semester to fulfill my fine arts requirement.",    "I just enrolled in CS 201 and I'm excited to start learning about algorithms.",    "I'm struggling in GYM 210, but I'm determined to improve my physical fitness.",    "This summer, I'm enrolled in BIO 301 and looking forward to learning about genetics.",    "I'm considering taking PE 201 next semester to fulfill my physical education requirement."]
sentences = [    "I woke up early this morning and went for a run.",    "Sheila is a great singer and songwriter.",    "The pizza at that new restaurant is amazing.",    "I can't wait for summer vacation to start.",    "The movie we saw last night was really funny.",    "My dog is always excited to see me when I come home.",    "I'm thinking about starting a new hobby, like painting or woodworking.",    "I'm really proud of my little brother for getting into college.",    "I love listening to music while I'm driving.",    "I have a lot of work to do, but I'm trying to stay focused.",    "I can't believe how fast time is flying by.",    "I'm going to the beach this weekend with some friends.",    "I'm trying to eat healthier and exercise more.",    "I'm thinking about taking a cooking class to improve my skills.",    "I'm really enjoying this book that I'm reading right now.",    "I'm trying to learn a new language, but it's difficult.",    "I'm excited to see what the future holds.",    "I'm grateful for all the support I've received from my family and friends.",    "I'm always looking for ways to improve myself and grow as a person."]


## Define our variables

In [ ]:
model = None
# output_dir=Path("C:\\Users\\nithi\\Documents\\ner")
output_dir=Path("NerModels")
n_iter=150

## Load the model

In [ ]:
if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

## Set up the pipeline

In [ ]:
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe('ner')
else:
    ner = nlp.get_pipe('ner')

## Train the Recognizer

In [ ]:
from spacy.training import Example

for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            example = Example.from_dict(nlp.make_doc(text), annotations)
            nlp.update(
                [example],  
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
        print(losses)

## Test the trained model

In [ ]:
for text, _ in TRAIN_DATA:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
#     print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

## Save the model

In [ ]:
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)        

## Test the saved model

In [ ]:
# print("Loading from", output_dir)
# nlp2 = spacy.load(output_dir)
# for text, _ in TEST_DATA:
#     doc = nlp2(text)
#     print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
#     print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

In [ ]:
# should be empty - no course name mentioned in set.
for sent in TEST_Sentences:
    doc = nlp2(sent)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
#     print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

In [ ]:
# should be empty - no course name mentioned in set.
for sent in sentences:
    doc = nlp2(sent)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
#     print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

## Clean version

In [ ]:
# from __future__ import unicode_literals, print_function
# import plac
# import random
from pathlib import Path
import spacy
# from tqdm import tqdm 

In [ ]:
TRAIN_DATA = [
    ('I am taking cosc 301', {
        'entities': [(12, 20, 'COR')]
    }),
    ('I am taking phil 331', {
        'entities': [(12, 20, 'COR')]
    }),
    ('I am taking data 301', {
        'entities': [(12, 20, 'COR')]
    }),
    ('I love engl 351 so much', {
        'entities': [(7, 15, "COR")]
    }),
    ('Why is span 419 so hard', {
        'entities': [(7, 15, "COR")]
    }),
    ('I just enrolled in math 101 and I am excited to start learning about calculus.', {
    'entities': [(20, 28, "COR")]
    }),
    ('Last semester, I took psyc 200 and it was one of my favorite classes.', {
        'entities': [(18, 26, "COR")]
    }),
    ('psyc 200 is one of my favorite classes.', {
        'entities': [(0, 7, "COR")]
    }),
    ('I am struggling in econ 301, but I am determined to improve my understanding of macroeconomics.', {
        'entities': [(19, 27, "COR")]
    }),
    ('I am considering taking chem 110 next semester.', {
        'entities': [(25, 33, "COR")]
    }),
    ('I have a final exam in stat 400 next week and I have been studying non-stop to prepare.', {
        'entities': [(22, 30, "COR")]
    }),
    ('I am struggling in mus 210, but I am determined to improve my musical abilities.', {
        'entities': [(19, 26, "COR")]
    }),
    
    # Computer Science
    ('I am majoring in computer science', {'entities': [(18, 34, 'MAJOR')]}),
    ('My major is computer science', {'entities': [(12, 28, 'MAJOR')]}),
#     ('I study computer science', {'entities': [(7, 23, 'MAJOR')]}),
#     ('Computer science is my field of study', {'entities': [(0, 17, 'MAJOR')]}),
    
    # Biology
    ('I am majoring in biology', {'entities': [(18, 25, 'MAJOR')]}),
    ('My major is biology', {'entities': [(12, 19, 'MAJOR')]}),
#     ('I study biology', {'entities': [(7, 14, 'MAJOR')]}),
#     ('Biology is my field of study', {'entities': [(0, 7, 'MAJOR')]}),
    
    # Data Science
#     ('I am majoring in data science', {'entities': [(18, 30, 'MAJOR')]}),
#     ('My major is data science', {'entities': [(12, 24, 'MAJOR')]}),
#     ('I study data science', {'entities': [(7, 19, 'MAJOR')]}),
#     ('Data science is my field of study', {'entities': [(0, 12, 'MAJOR')]}),
    
    # Mechanical Engineering
    ('I am majoring in mechanical engineering', {'entities': [(18, 40, 'MAJOR')]}),
    ('My major is mechanical engineering', {'entities': [(12, 34, 'MAJOR')]}),
#     ('I study mechanical engineering', {'entities': [(7, 32, 'MAJOR')]}),
#     ('Mechanical engineering is my field of study', {'entities': [(0, 23, 'MAJOR')]}),
    
    # Physics
    ('I am majoring in physics', {'entities': [(18, 25, 'MAJOR')]}),
    ('My major is physics', {'entities': [(12, 19, 'MAJOR')]}),
#     ('I study physics', {'entities': [(7, 14, 'MAJOR')]}),
#     ('Physics is my field of study', {'entities': [(0, 7, 'MAJOR')]}),
    
    # Psychology
    ('I am majoring in psychology', {'entities': [(18, 28, 'MAJOR')]}),
    ('My major is psychology', {'entities': [(12, 22, 'MAJOR')]}),
#     ('I study psychology', {'entities': [(7, 17, 'MAJOR')]}),
#     ('Psychology is my field of study', {'entities': [(0, 10, 'MAJOR')]}),
    
    # Business
#     ('I am majoring in business', {'entities': [(18, 25, 'MAJOR')]}),
#     ('My major is business', {'entities': [(12, 19, 'MAJOR')]}),
#     ('I study business', {'entities': [(7, 14, 'MAJOR')]}),
#     ('Business is my field of study', {'entities': [(0, 7, 'MAJOR')]}),
    
    # Chemistry
    ('I am majoring in chemistry', {'entities': [(18, 27, 'MAJOR')]}),
    ('My major is chemistry', {'entities': [(12, 21, 'MAJOR')]}),
    ('I study chemistry', {'entities': [(7, 16, 'MAJOR')]}),
    ('Chemistry is my field of study', {'entities': [(0, 9, 'MAJOR')]}),

    # Mathematics
    ('I am majoring in mathematics', {'entities': [(18, 29, 'MAJOR')]}),
    ('My major is mathematics', {'entities': [(12, 23, 'MAJOR')]}),
#     ('I study mathematics', {'entities': [(7, 18, 'MAJOR')]}),
#     ('Mathematics is my field of study', {'entities': [(0, 12, 'MAJOR')]}),

    # English
#     ('I am majoring in English', {'entities': [(18, 24, 'MAJOR')]}),
#     ('My major is English', {'entities': [(12, 18, 'MAJOR')]}),
#     ('I study English', {'entities': [(7, 13, 'MAJOR')]}),
#     ('English is my field of study', {'entities': [(0, 7, 'MAJOR')]}),

    # Environmental Science
#     ('I am majoring in environmental science', {'entities': [(18, 39, 'MAJOR')]}),
#     ('My major is environmental science', {'entities': [(12, 33, 'MAJOR')]}),
#     ('I study environmental science', {'entities': [(7, 28, 'MAJOR')]}),
#     ('Environmental science is my field of study', {'entities': [(0, 22, 'MAJOR')]}),

    # History
#     ('I am majoring in history', {'entities': [(18, 25, 'MAJOR')]}),
#     ('My major is history', {'entities': [(12, 19, 'MAJOR')]}),
#     ('I study history', {'entities': [(7, 14, 'MAJOR')]}),
#     ('History is my field of study', {'entities': [(0, 7, 'MAJOR')]})
]

TEST_DATA = [
     ('How is math 101?', {
        'entities': [(7, 15, 'PER')]
    }),
     ('Who is cosc 554?', {
        'entities': [(7, 19, 'PER')]
    }),
    ('I like London and Berlin.', {
        'entities': [(7, 13, 'LOC'), (18, 24, 'LOC')]
    }),
    ('what is phil 331 about', {
        'entities': [(12, 20, 'COR')]
    })
]

TEST_Sentences = ["I am taking COSC 301", "I'm taking PSYC 210 and learning about the different approaches to psychology.",    "I just enrolled in math 101 and I'm excited to start learning about calculus.",    "I'm considering taking MATH 202 next semester, but I'm not sure if I'm ready for the challenge.",    "Last semester, I took PSYC 200 and it was one of my favorite classes.",    "I'm struggling in ECON 301, but I'm determined to improve my understanding of macroeconomics.",    "This summer, I'm enrolled in SPAN 301 and looking forward to improving my language skills.",    "I have a final exam in STAT 400 next week and I have been studying non-stop to prepare.",    "I loved taking HIST 205 and learning about the Civil Rights Movement.",    "I am considering taking PHIL 331 next semester to fulfill my ethics requirement.", "I'm taking ART 101 this semester to fulfill my fine arts requirement.",    "I just enrolled in CS 201 and I'm excited to start learning about algorithms.",    "I'm struggling in GYM 210, but I'm determined to improve my physical fitness.",    "This summer, I'm enrolled in BIO 301 and looking forward to learning about genetics.",    "I'm considering taking PE 201 next semester to fulfill my physical education requirement."]
sentences = [    "I woke up early this morning and went for a run.",    "Sheila is a great singer and songwriter.",    "The pizza at that new restaurant is amazing.",    "I can't wait for summer vacation to start.",    "The movie we saw last night was really funny.",    "My dog is always excited to see me when I come home.",    "I'm thinking about starting a new hobby, like painting or woodworking.",    "I'm really proud of my little brother for getting into college.",    "I love listening to music while I'm driving.",    "I have a lot of work to do, but I'm trying to stay focused.",    "I can't believe how fast time is flying by.",    "I'm going to the beach this weekend with some friends.",    "I'm trying to eat healthier and exercise more.",    "I'm thinking about taking a cooking class to improve my skills.",    "I'm really enjoying this book that I'm reading right now.",    "I'm trying to learn a new language, but it's difficult.",    "I'm excited to see what the future holds.",    "I'm grateful for all the support I've received from my family and friends.",    "I'm always looking for ways to improve myself and grow as a person."]


In [ ]:
model = None
output_dir=Path("NerModels")

In [ ]:
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
for sent in TEST_Sentences:
    doc = nlp2(sent)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
#     print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

In [ ]:
# should be empty - no course name mentioned in set.
for sent in sentences:
    doc = nlp2(sent)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
#     print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

In [ ]:
doc = nlp2("do i have to take cosc 121 if i am majoring in computer science")

In [ ]:
print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

In [ ]:
print(doc.ents[0].label_)

In [ ]:
# for ent in doc.ents:
dict = {}

# dict.update({doc.ents[0].label_ : doc.ents[0].text})

In [ ]:
dict

In [ ]:
dict = {}
for ent in doc.ents:
    dict.update({ent.label_: ent.text})

print(dict['COR'], dict['MAJOR'])

In [ ]:
dict

In [ ]:
from pathlib import Path
import spacy

output_dir=Path("NerModels")

# print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)

doc = nlp2("cosc 111  is computer science")
# print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
user = {}
for ent in doc.ents:
    user.update({ent.label_: ent.text})

direct = {"computer science": ["cosc 121", "math 100", "math 101", "cosc 211", "cosc 221", "cosc 222", "math 221", "stat 230", "cosc 320", "cosc 304", "cosc 310" "cosc 341" "cosc 499", "phil 331"]}
optional = {"computer science": [["cosc 111", "cosc 123"], {"engl 109": ["2", "engl 112", "engl 113", "engl 114", "engl 150", "engl 151", "engl 153", "engl 154","engl 155", "engl 156", "engl 203", "corh 203", "corh 205", "apsc 176", "apsc 201"]}, ["phys 111", "phys 112"]]}
reply = ""

if user['COR'] in direct[user['MAJOR']]:
    reply = "Yes " + user['COR'] + " is a requirement for " + user['MAJOR'] + "." 
if user['COR'] in optional[user["MAJOR"]]:
    reply = "Yes " + user['COR']

for type in optional[user['MAJOR']]:
    if isinstance(type, dict):
        l=[]
        [l.extend([k,v]) for k,v in type.items()]
        y = [l[0]]
        [y.extend(l[1])]
#         print(y)
        if user['COR'] in y:
            reply = "Yes, take one of "+ y[0] + " or " + y[1] + " of "  
            del y[0]
            del y[0]
            for i in y:
                reply += i + ", "
    else:
        if user['COR'] in type:
            reply = "Yes, take one of "+ type[0] + " or " + type[1]  
    
if reply == "":
    reply = user['COR'] + " is not a requirement for " + user['MAJOR'] + " but might be used as an elective, speak with an Academic & Career Advisor for more clarity."
    
print(reply)

In [ ]:
optional = {"computer science": [["cosc 111", "cosc 123"], {"engl 109": ["2", "engl 112", "engl 113", "engl 114", "engl 150", "engl 151", "engl 153", "engl 154","engl 155", "engl 156", "engl 203", "corh 203", "corh 205", "apsc 176", "apsc 201"]}, ["phys 111", "phys 112"]]}

In [ ]:
bar = {"engl 109": ["2", "engl 112", "engl 113", "engl 114", "engl 150", "engl 151", "engl 153", "engl 154","engl 155", "engl 156", "engl 203", "corh 203", "corh 205", "apsc 176", "apsc 201"]}

l=[]
[l.extend x for x in bar.items()]
print(l)